# Multiple Outputs
>  In this chapter, you will build neural networks with multiple outputs, which can be used to solve regression problems with multiple targets. You will also build a model that solves a regression problem and a classification problem simultaneously.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp, marked]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 4 exercises "Advanced Deep Learning with Keras" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (8, 8)

In [2]:
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Subtract
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Two-output models

### Simple two-output model

<div class=""><p>In this exercise, you will use the tournament data to build one model that makes two predictions: the scores of both teams in a given game. Your inputs will be the seed difference of the two teams, as well as the predicted score difference from the model you built in chapter 3.</p>
<p>The output from your model will be the predicted score for team 1 as well as team 2.  This is called "multiple target regression": one model making more than one prediction.</p></div>

Instructions
<ul>
<li>Create a single input layer with 2 columns.</li>
<li>Connect this input to a Dense layer with 2 units.</li>
<li>Create a model with <code>input_tensor</code> as the input and <code>output_tensor</code> as the output.</li>
<li>Compile the model with <code>'adam'</code> as the optimizer and <code>'mean_absolute_error'</code> as the loss function.</li>
</ul>

In [12]:
# Define the input
input_tensor = Input((2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

### Fit a model with two outputs

<div class=""><p>Now that you've defined your 2-output model, fit it to the tournament data.  I've split the data into <code>games_tourney_train</code> <code>and games_tourney_test</code>, so use the training set to fit for now.</p>
<p>This model will use the pre-tournament seeds, as well as your pre-tournament predictions from the regular season model you built previously in this course.</p>
<p>As a reminder, this model will predict the scores of both teams.</p></div>

In [5]:
games_tourney_test = pd.read_csv('https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/17-advanced-deep-learning-with-keras/datasets/games_tourney_test.csv')
games_tourney_train = pd.read_csv('https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/17-advanced-deep-learning-with-keras/datasets/games_tourney_train.csv')

Instructions
<ul>
<li>Fit the model to the <code>games_tourney_train</code> dataset using 100 epochs and a batch size of 16384.</li>
<li>The input columns are <code>'seed_diff'</code>, and <code>'pred'</code>.</li>
<li>The target columns are <code>'score_1'</code> and <code>'score_2'</code>.</li>
</ul>

In [13]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
  		  games_tourney_train[['score_1', 'score_2']],
  		  verbose=False,
  		  epochs=100,
  		  batch_size=1) #16384

### Inspect the model (I)

<div class=""><p>Now that you've fit your model, let's take a look at it. You can use the <code>.get_weights()</code> method to inspect your model's weights.</p>
<p>The input layer will have 4 weights: 2 for each input times 2 for each output.</p>
<p>The output layer will have 2 weights, one for each output.</p></div>

Instructions
<ul>
<li>Print the <code>model</code>'s weights.</li>
<li>Print the column means of the training data (<code>games_tourney_train</code>).</li>
</ul>

In [14]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[ 0.16611078, -0.15395507],
       [ 0.33728385, -0.32625207]], dtype=float32), array([70.89105, 70.89808], dtype=float32)]
season        1.998074e+03
team_1        5.556771e+03
team_2        5.556771e+03
home          0.000000e+00
seed_diff     0.000000e+00
score_diff    0.000000e+00
score_1       7.162128e+01
score_2       7.162128e+01
won           5.000000e-01
pred         -1.624447e-14
dtype: float64


**Did you notice that both output weights are about ~72? This is because, on average, a team will score about 72 points in the tournament.**

### Evaluate the model

<p>Now that you've fit your model and inspected it's weights to make sure it makes sense, evaluate it on the tournament test set to see how well it performs on new data.</p>

Instructions
<ul>
<li>Evaluate the model on <code>games_tourney_test</code>.</li>
<li>Use the same inputs and outputs as the training set.</li>
</ul>

In [15]:
# Evaluate the model on the tournament test data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']], games_tourney_test[['score_1', 'score_2']], verbose=False))

8.669510841369629


**This model is pretty accurate at predicting tournament scores!**

## Single model for classification and regression

### Classification and regression in one model

<div class=""><p>Now you will create a different kind of 2-output model.  This time, you will predict the score difference, instead of both team's scores and then you will predict the probability that team 1 won the game.  This is a pretty cool model: it is going to do both classification and regression!</p>
<p>In this model, turn off the bias, or intercept for each layer.  Your inputs (seed difference and predicted score difference) have a mean of very close to zero, and your outputs both have means that are close to zero, so your model shouldn't need the bias term to fit the data well.</p></div>

Instructions
<ul>
<li>Create a single input layer with 2 columns.</li>
<li>The first output layer should have 1 unit with <code>'linear'</code> activation and no bias term.</li>
<li>The second output layer should have 1 unit with <code>'sigmoid'</code> activation and no bias term. Also, use the first output layer as an input to this layer.</li>
<li>Create a model with these input and outputs.</li>
</ul>

In [28]:
# Create an input layer with 2 columns
input_tensor = Input((2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

**This kind of model is only possible with a neural network.**

### Compile and fit the model

<div class=""><p>Now that you have a model with 2 outputs, compile it with 2 loss functions: mean absolute error (MAE) for <code>'score_diff'</code> and binary cross-entropy (also known as logloss) for <code>'won'</code>. Then fit the model with <code>'seed_diff'</code> and <code>'pred'</code> as inputs. For outputs, predict <code>'score_diff'</code> and <code>'won'</code>.</p>
<p>This model can use the scores of the games to make sure that close games (small score diff) have lower win probabilities than blowouts (large score diff).</p>
<p>The regression problem is easier than the classification problem because MAE punishes the model less for a loss due to random chance. For example, if <code>score_diff</code> is -1 and <code>won</code> is 0, that means <code>team_1</code> had some bad luck and lost by a single free throw. The data for the easy problem helps the model find a solution to the hard problem.</p></div>

Instructions
<ul>
<li>Import <code>Adam</code> from <code>keras.optimizers</code>.</li>
<li>Compile the model with 2 losses: <code>'mean_absolute_error'</code> and <code>'binary_crossentropy'</code>, and use the Adam optimizer with a learning rate of 0.01.</li>
<li>Fit the model with <code>'seed_diff'</code> and <code>'pred'</code> columns as the inputs and <code>'score_diff'</code> and <code>'won'</code> columns as the targets.</li>
<li>Use 10 epochs and a batch size of 16384.</li>
</ul>

In [29]:
# Import the Adam optimizer
from keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(lr=0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=20,
          verbose=True,
          batch_size=16) #16384

Epoch 1/20
215/215 [==============================] - 1s 1ms/step - loss: 10.7625 - dense_5_loss: 10.0610 - dense_6_loss: 0.7015
Epoch 2/20
215/215 [==============================] - 0s 1ms/step - loss: 9.2833 - dense_5_loss: 8.7435 - dense_6_loss: 0.5398
Epoch 3/20
215/215 [==============================] - 0s 1ms/step - loss: 9.3638 - dense_5_loss: 8.8367 - dense_6_loss: 0.5272
Epoch 4/20
215/215 [==============================] - 0s 1ms/step - loss: 9.1804 - dense_5_loss: 8.6547 - dense_6_loss: 0.5257
Epoch 5/20
215/215 [==============================] - 0s 1ms/step - loss: 9.1771 - dense_5_loss: 8.6502 - dense_6_loss: 0.5269
Epoch 6/20
215/215 [==============================] - 0s 1ms/step - loss: 9.1999 - dense_5_loss: 8.6800 - dense_6_loss: 0.5199
Epoch 7/20
215/215 [==============================] - 0s 1ms/step - loss: 9.3781 - dense_5_loss: 8.8504 - dense_6_loss: 0.5277
Epoch 8/20
215/215 [==============================] - 0s 1ms/step - loss: 9.1810 - dense_5_loss: 8.6574 - den

**You just fit a model that is both a classifier and a regressor!**

### Inspect the model (II)

<p>Now you should take a look at the weights for this model. In particular, note the last weight of the model. This weight converts the predicted score difference to a predicted win probability. If you multiply the predicted score difference by the last weight of the model and then apply the sigmoid function, you get the win probability of the game.</p>

Instructions 1/2
<ul>
<li>Print the <code>model</code>'s weights.</li>
<li>Print the column means of the training data (<code>games_tourney_train</code>).</li>
</ul>

In [30]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[0.30786675],
       [0.6838873 ]], dtype=float32), array([[0.10927454]], dtype=float32)]
season        1.998074e+03
team_1        5.556771e+03
team_2        5.556771e+03
home          0.000000e+00
seed_diff     0.000000e+00
score_diff    0.000000e+00
score_1       7.162128e+01
score_2       7.162128e+01
won           5.000000e-01
pred         -1.624447e-14
dtype: float64


Instructions 2/2
<ul>
<li>Print the approximate win probability predicted for a close game (1 point difference).</li>
<li>Print the approximate win probability predicted blowout game (10 point difference).</li>
</ul>

In [31]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


In [34]:
model.get_weights()[1]

array([[0.10927454]], dtype=float32)

In [35]:
# Print the approximate win probability predicted close game
print(sigmoid(1 * model.get_weights()[1]))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * model.get_weights()[1]))

[[0.5272915]]
[[0.7488983]]


**So sigmoid(1 * 0.14) is 0.53, which represents a pretty close game and sigmoid(10 * 0.14) is 0.80, which represents a pretty likely win. In other words, if the model predicts a win of 1 point, it is less sure of the win than if it predicts 10 points. Who says neural networks are black boxes?**

### Evaluate on new data with two metrics

<div class=""><p>Now that you've fit your model and inspected its weights to make sure they make sense, evaluate your model on the tournament test set to see how well it does on new data.</p>
<p>Note that in this case, Keras will return 3 numbers: the first number will be the sum of both the loss functions, and then the next 2 numbers will be the loss functions you used when defining the model.</p>
<p>Ready to take your deep learning to the next level? Check out <a href="https://www.datacamp.com/courses/convolutional-neural-networks-for-image-processing" target="_blank" rel="noopener noreferrer">"Convolutional Neural Networks for Image Processing"</a>.</p></div>

Instructions
<ul>
<li>Evaluate the model on <code>games_tourney_test</code>.</li>
<li>Use the same inputs and outputs as the training set.</li>
</ul>

In [37]:
# Evaluate the model on new data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']],
               [games_tourney_test[['score_diff']], games_tourney_test[['won']]], verbose=False))

[9.438961029052734, 8.883465766906738, 0.5554943084716797]
